# Merge all Hospital Archives dataframes

In [1]:
import pandas as pd
import numpy as np
import warnings
from IPython.utils import io
import sys
import csv
import os

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

main_dir = os.path.expanduser("~/GitHub/hospitals-data-archive/")


## Load curated/compiled hospital archives data

In [3]:
ls2 = [#'HVBP Total Performance',
       #'HVBP Clinical Outcomes',
       #'HVBP Efficiency',
       #'HVBP Safety',
       #'HVBP HCAHPS',
       'Outpatient Imaging Efficiency',
       'Payment and Value of Care',
       'Complications and Deaths',
       'Hospital Readmissions Reduction Program',
       'Hospital Acquired Conditions Reduction Program',
       #'HCAHPS',
       'Timely and Effective Care',
       'Unplanned Visits',
]


In [4]:
df = pd.read_pickle(main_dir + '/dataframes/partial_dataframes/hai_df.pkl.gz', compression='gzip')
print('df.shape:', df.shape)
for i in list(df):
    if i not in ['Facility ID', 'Facility Name', 'file_month','file_year']:
        df.rename(columns={i: 'Healthcare Associated Infections: ' + i}, inplace=True)

ls1 = [#'hvbp_total_performance_scores_df.pkl.gz', 
       #'hvbp_clinical_outcomes_df.pkl.gz', 
       #'hvbp_efficiency_df.pkl.gz',
       #'hvbp_safety_df.pkl.gz', 
       #'hvbp_hcahps_df.pkl.gz',
       'Outpatient_Imaging_Efficiency_df.pkl.gz', 
       'Payment_and_ValueOfCare_df.pkl.gz', 
       'complication_and_deaths_df.pkl.gz', 
       'hrrp_df.pkl.gz',
       'hacrp_df.pkl.gz',
       #'hcahps_df.pkl.gz',
       'Timely_and_Effective_Care_df.pkl.gz',
       'unplanned_visits_df.pkl.gz',
      ]

brk = 0
for i, l in enumerate(ls1):
    print(l)
    tdf = pd.read_pickle(main_dir + '/dataframes/partial_dataframes/' + l, compression='gzip')
    
    for ii in list(tdf):
        if ii not in ['Facility ID', 'Facility Name', 'file_month','file_year']:
            tdf.rename(columns={ii: ls2[i] + ': ' + ii}, inplace=True)
    
    df = df.merge(tdf, how='outer')
    print('df.shape:', df.shape)
    
    for iii in list(df):
        if '_x' in iii:
            print('redundant column name:', iii)
            brk = 1
            break
            
    if brk == 1: break
    del tdf

print('df.shape:', df.shape)
df.head()


df.shape: (163645, 54)
Outpatient_Imaging_Efficiency_df.pkl.gz
df.shape: (177692, 61)
Payment_and_ValueOfCare_df.pkl.gz
df.shape: (182213, 77)
complication_and_deaths_df.pkl.gz
df.shape: (503345, 105)
hrrp_df.pkl.gz
df.shape: (524065, 135)
hacrp_df.pkl.gz
df.shape: (530106, 156)
Timely_and_Effective_Care_df.pkl.gz
df.shape: (530242, 308)
unplanned_visits_df.pkl.gz
df.shape: (530242, 373)
df.shape: (530242, 373)


,Facility ID,Facility Name,file_month,file_year,Healthcare Associated Infections: CAUTI (SIR) (HAI-2 SIR) (Score),Healthcare Associated Infections: CAUTI Number of Procedures (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2 NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2a NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2 ELIG CASES) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2a ELIG CASES) (Score),Healthcare Associated Infections: CAUTI SIR (ICUs only) (HAI-2a SIR) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2a DOPC) (Score),Healthcare Associated Infections: CAUTI lower CL (HAI-2 CI LOWER) (Score),Healthcare Associated Infections: CAUTI lower CL (HAI-2a CI LOWER) (Score),Healthcare Associated Infections: CAUTI upper CL (HAI-2 CI UPPER) (Score),Healthcare Associated Infections: CAUTI upper CL (HAI-2a CI UPPER) (Score),Healthcare Associated Infections: CDIFF (SIR) (HAI-6 SIR) (Score),Healthcare Associated Infections: CDIFF Observed Cases (HAI-6 NUMERATOR) (Score),Healthcare Associated Infections: CDIFF Predicted Cases (HAI-6 ELIG CASES) (Score),Healthcare Associated Infections: CDIFF lower CL (HAI-6 CI LOWER) (Score),Healthcare Associated Infections: CDIFF patient days (HAI-6 DOPC) (Score),Healthcare Associated Infections: CDIFF upper CL (HAI-6 CI UPPER) (Score),Healthcare Associated Infections: CLABSI (SIR) (HAI-1 SIR) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1a DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Procedures (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1 NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1a NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1 ELIG CASES) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1a ELIG CASES) (Score),Healthcare Associated Infections: CLABSI SIR (ICUs only) (HAI-1a SIR) (Score),Healthcare Associated Infections: CLABSI lower CL (HAI-1 CI LOWER) (Score),Healthcare Associated Infections: CLABSI lower CL (HAI-1a CI LOWER) (Score),Healthcare Associated Infections: CLABSI upper CL (HAI-1 CI UPPER) (Score),Healthcare Associated Infections: CLABSI upper CL (HAI-1a CI UPPER) (Score),Healthcare Associated Infections: MRSA (SIR) (HAI-5 SIR) (Score),Healthcare Associated Infections: MRSA Observed Cases (HAI-5 NUMERATOR) (Score),Healthcare Associated Infections: MRSA Predicted Cases (HAI-5 ELIG CASES) (Score),Healthcare Associated Infections: MRSA lower CL (HAI-5 CI LOWER) (Score),Healthcare Associated Infections: MRSA patient days (HAI-5 DOPC) (Score),Healthcare Associated Infections: MRSA upper CL (HAI-5 CI UPPER) (Score),Healthcare Associated Infections: SSI Abdominal Hysterectomy (SIR) (HAI-4 SIR) (Score),Healthcare Associated Infections: SSI Abdominal Number of Procedures (HAI-4 DOPC) (Score),Healthcare Associated Infections: SSI Abdominal Observed Cases (HAI-4 NUMERATOR) (Score),Healthcare Associated Infections: SSI Abdominal Predicted Cases (HAI-4 ELIG CASES) (Score),Healthcare Associated Infections: SSI Abdominal lower CL (HAI-4 CI LOWER) (Score),Healthcare Associated Infections: SSI Abdominal upper CL (HAI-4 CI UPPER) (Score),Healthcare Associated Infections: SSI Colon Number of Procedures (HAI-3 DOPC) (Score),Healthcare Associated Infections: SSI Colon Observed Cases (HAI-3 NUMERATOR) (Score),Healthcare Associated Infections: SSI Colon Predicted Cases (HAI-3 ELIG CASES) (Score),Healthcare Associated Infections: SSI Colon Surgery (SIR) (HAI-3 SIR) (Score),Healthcare Associated Infections: SSI Colon lower CL (HAI-3 CI LOWER) (Score),Healthcare Associated Infections: SSI Colon upper CL (HAI-3 CI UPP

In [5]:
tdf = df[df['Facility ID'] == '010001']
tdf['marker'] = tdf['Facility ID'] + ' | ' + tdf['Facility Name']  + ' | ' + tdf['file_month'] + ' | ' + tdf['file_year']
print(tdf.shape)
column = tdf.pop('marker')
tdf.insert(0, column.name, column)
tdf.drop_duplicates(inplace=True)
print(tdf.shape)

print(' ', len(tdf['marker']))
print(' ', len(tdf['marker'].unique()))
markers = tdf['marker'].tolist()
u_markers = tdf['marker'].unique().tolist()
i = str()
for i in u_markers:
    if markers.count(i) > 1:
        print(i)
        break

tdf = tdf[tdf['marker'] == i]
tdf.head(10)

(129, 374)
(129, 374)
  129
  49
010001 | SOUTHEAST ALABAMA MEDICAL CENTER | 01 | 2015


,marker,Facility ID,Facility Name,file_month,file_year,Healthcare Associated Infections: CAUTI (SIR) (HAI-2 SIR) (Score),Healthcare Associated Infections: CAUTI Number of Procedures (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2 NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2a NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2 ELIG CASES) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2a ELIG CASES) (Score),Healthcare Associated Infections: CAUTI SIR (ICUs only) (HAI-2a SIR) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2a DOPC) (Score),Healthcare Associated Infections: CAUTI lower CL (HAI-2 CI LOWER) (Score),Healthcare Associated Infections: CAUTI lower CL (HAI-2a CI LOWER) (Score),Healthcare Associated Infections: CAUTI upper CL (HAI-2 CI UPPER) (Score),Healthcare Associated Infections: CAUTI upper CL (HAI-2a CI UPPER) (Score),Healthcare Associated Infections: CDIFF (SIR) (HAI-6 SIR) (Score),Healthcare Associated Infections: CDIFF Observed Cases (HAI-6 NUMERATOR) (Score),Healthcare Associated Infections: CDIFF Predicted Cases (HAI-6 ELIG CASES) (Score),Healthcare Associated Infections: CDIFF lower CL (HAI-6 CI LOWER) (Score),Healthcare Associated Infections: CDIFF patient days (HAI-6 DOPC) (Score),Healthcare Associated Infections: CDIFF upper CL (HAI-6 CI UPPER) (Score),Healthcare Associated Infections: CLABSI (SIR) (HAI-1 SIR) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1a DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Procedures (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1 NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1a NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1 ELIG CASES) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1a ELIG CASES) (Score),Healthcare Associated Infections: CLABSI SIR (ICUs only) (HAI-1a SIR) (Score),Healthcare Associated Infections: CLABSI lower CL (HAI-1 CI LOWER) (Score),Healthcare Associated Infections: CLABSI lower CL (HAI-1a CI LOWER) (Score),Healthcare Associated Infections: CLABSI upper CL (HAI-1 CI UPPER) (Score),Healthcare Associated Infections: CLABSI upper CL (HAI-1a CI UPPER) (Score),Healthcare Associated Infections: MRSA (SIR) (HAI-5 SIR) (Score),Healthcare Associated Infections: MRSA Observed Cases (HAI-5 NUMERATOR) (Score),Healthcare Associated Infections: MRSA Predicted Cases (HAI-5 ELIG CASES) (Score),Healthcare Associated Infections: MRSA lower CL (HAI-5 CI LOWER) (Score),Healthcare Associated Infections: MRSA patient days (HAI-5 DOPC) (Score),Healthcare Associated Infections: MRSA upper CL (HAI-5 CI UPPER) (Score),Healthcare Associated Infections: SSI Abdominal Hysterectomy (SIR) (HAI-4 SIR) (Score),Healthcare Associated Infections: SSI Abdominal Number of Procedures (HAI-4 DOPC) (Score),Healthcare Associated Infections: SSI Abdominal Observed Cases (HAI-4 NUMERATOR) (Score),Healthcare Associated Infections: SSI Abdominal Predicted Cases (HAI-4 ELIG CASES) (Score),Healthcare Associated Infections: SSI Abdominal lower CL (HAI-4 CI LOWER) (Score),Healthcare Associated Infections: SSI Abdominal upper CL (HAI-4 CI UPPER) (Score),Healthcare Associated Infections: SSI Colon Number of Procedures (HAI-3 DOPC) (Score),Healthcare Associated Infections: SSI Colon Observed Cases (HAI-3 NUMERATOR) (Score),Healthcare Associated Infections: SSI Colon Predicted Cases (HAI-3 ELIG CASES) (Score),Healthcare Associated Infections: SSI Colon Surgery (SIR) (HAI-3 SIR) (Score),Healthcare Associated Infections: SSI Colon lower CL (HAI-3 CI LOWER) (Score),Healthcare Associated Infections: SSI Colon upper CL (HAI-3

## Load general hospital info file from hospitals archives

In [6]:
hosp_df = pd.read_pickle(main_dir + '/dataframes/partial_dataframes/hospital_info_df.pkl.gz', compression='gzip')
hosp_df = hosp_df.filter(items=['Facility ID', 'Hospital Ownership', 'Hospital Type', 'file_month', 'file_year'])
print(hosp_df.shape)
hosp_df.drop_duplicates(inplace=True)
print(hosp_df.shape)

d = {'Acute Care Hospitals': 'Acute Care', 
     np.nan: 'Not given',  
     'Critical Access Hospitals': 'Critical Access', 
     'Acute Care - Department of Defense': 'Acute Care (DOD)',  
     'ACUTE CARE - VETERANS ADMINISTRATION': 'Acute Care (VA)', 
     'Acute Care - Veterans Administration': 'Acute Care (VA)'}
hosp_df['Hospital Type'].replace(to_replace=d, inplace=True)
print(hosp_df['Hospital Type'].unique().tolist(), '\n')

d = {'Veterans Health Administration': 'VA', 
     'Government Federal': 'Government - Federal', 
     'Government-State': 'Government - State', 
     'Voluntary non-profit-Other': 'Voluntary non-profit - Other', 
     'Voluntary non-profit-Private': 'Voluntary non-profit - Private', 
     'Government-Federal': 'Government - Federal'}
hosp_df['Hospital Ownership'].replace(to_replace=d, inplace=True)
print(hosp_df['Hospital Ownership'].unique().tolist(), '\n')

print(hosp_df.shape)
hosp_df = hosp_df[hosp_df['Facility ID'].isin(df['Facility ID'].unique())]
print(hosp_df.shape)

print(len(hosp_df['Facility ID'].unique()), 'unique hospital IDs')     
hosp_df.sort_values(by=['Facility ID', 'file_month', 'file_year'], ascending=False, inplace=True)

print(hosp_df.shape)
hosp_df.drop_duplicates(subset=['Facility ID'], keep='first', inplace=True)
hosp_df.drop(labels=['file_month', 'file_year'], axis=1, inplace=True)
print(hosp_df.shape)
hosp_df.head()

(231358, 5)
(231358, 5)
['Acute Care', 'Critical Access', 'Childrens', 'Psychiatric', 'Acute Care (DOD)', 'Acute Care (VA)'] 

['Government - Hospital District or Authority', 'Proprietary', 'Voluntary non-profit - Private', 'Government - State', 'Voluntary non-profit - Other', 'Government - Local', 'Voluntary non-profit - Church', 'Government - Federal', 'Tribal', 'Department of Defense', 'Physician', 'VA', 'Government'] 

(231358, 5)
(220030, 5)
5434 unique hospital IDs
(220030, 5)
(5434, 3)


,Facility ID,Hospital Ownership,Hospital Type
5445,670319,Voluntary non-profit - Private,Acute Care
5444,670314,Proprietary,Acute Care
5443,670310,Physician,Acute Care
5442,670309,Proprietary,Acute Care
5441,670300,Voluntary non-profit - Private,Acute Care


## Load file from HCRIS project that contains Lat's, Lon's, and numbers of beds

In [7]:
tdf2 = pd.read_pickle(main_dir + 'Files_For_App/GenDat4App.pkl')
tdf2.reset_index(drop=True, inplace=True)
tdf2.columns = tdf2.columns.droplevel([0,1,2])
tdf2.rename(columns={'Type of Control of Hospital (See Table I) (S2_1_C1_21)': 'Hospital Ownership',
                     'Hospital type (modified)': 'Hospital Type',
                     'Hospital Provider Number (PRVDR_NUM)': 'Facility ID',
                     'Hospital State (S2_1_C2_2)': 'State',
                     'Total Facility (S3_1_C2_27)': 'Beds',
                    }, inplace=True)

tdf2 = tdf2.filter(items=['Facility ID', 'Lat', 'Lon', 'Beds', 'State'], axis=1)
tdf2['Beds'] = tdf2.groupby(['Facility ID'])['Beds'].transform('mean')
tdf2['Beds'] = np.round(tdf2['Beds'], 0)

print(tdf2.shape)
tdf2.drop_duplicates(inplace=True)
print(tdf2.shape)
hosp_ids = df['Facility ID'].unique()
tdf2 = tdf2[tdf2['Facility ID'].isin(hosp_ids)]
print(tdf2.shape)
tdf2.head()

(73087, 5)
(6908, 5)
(5240, 5)


,Facility ID,Lat,Lon,Beds,State
0,010001,31.214058,-85.361725,377.0,AL
12,010005,NaN,NaN,193.0,AL
24,010006,34.802756,-87.652191,314.0,AL
36,010007,31.291972,-86.255415,64.0,AL
48,010008,31.692595,-86.266156,49.0,AL


## Merge dataframes

In [8]:
tdf = df.copy(deep=True)

print(tdf.shape)
tdf = tdf.merge(hosp_df, how='outer', on=['Facility ID'])
print(tdf.shape)
tdf = tdf.merge(tdf2, how='outer', on=['Facility ID'])
print(tdf.shape)
tdf.drop_duplicates(inplace=True)
print(tdf.shape)
tdf.head()

(530242, 373)
(530242, 375)
(530242, 379)
(530242, 379)


,Facility ID,Facility Name,file_month,file_year,Healthcare Associated Infections: CAUTI (SIR) (HAI-2 SIR) (Score),Healthcare Associated Infections: CAUTI Number of Procedures (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2 NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2a NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2 ELIG CASES) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2a ELIG CASES) (Score),Healthcare Associated Infections: CAUTI SIR (ICUs only) (HAI-2a SIR) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2a DOPC) (Score),Healthcare Associated Infections: CAUTI lower CL (HAI-2 CI LOWER) (Score),Healthcare Associated Infections: CAUTI lower CL (HAI-2a CI LOWER) (Score),Healthcare Associated Infections: CAUTI upper CL (HAI-2 CI UPPER) (Score),Healthcare Associated Infections: CAUTI upper CL (HAI-2a CI UPPER) (Score),Healthcare Associated Infections: CDIFF (SIR) (HAI-6 SIR) (Score),Healthcare Associated Infections: CDIFF Observed Cases (HAI-6 NUMERATOR) (Score),Healthcare Associated Infections: CDIFF Predicted Cases (HAI-6 ELIG CASES) (Score),Healthcare Associated Infections: CDIFF lower CL (HAI-6 CI LOWER) (Score),Healthcare Associated Infections: CDIFF patient days (HAI-6 DOPC) (Score),Healthcare Associated Infections: CDIFF upper CL (HAI-6 CI UPPER) (Score),Healthcare Associated Infections: CLABSI (SIR) (HAI-1 SIR) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1a DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Procedures (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1 NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1a NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1 ELIG CASES) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1a ELIG CASES) (Score),Healthcare Associated Infections: CLABSI SIR (ICUs only) (HAI-1a SIR) (Score),Healthcare Associated Infections: CLABSI lower CL (HAI-1 CI LOWER) (Score),Healthcare Associated Infections: CLABSI lower CL (HAI-1a CI LOWER) (Score),Healthcare Associated Infections: CLABSI upper CL (HAI-1 CI UPPER) (Score),Healthcare Associated Infections: CLABSI upper CL (HAI-1a CI UPPER) (Score),Healthcare Associated Infections: MRSA (SIR) (HAI-5 SIR) (Score),Healthcare Associated Infections: MRSA Observed Cases (HAI-5 NUMERATOR) (Score),Healthcare Associated Infections: MRSA Predicted Cases (HAI-5 ELIG CASES) (Score),Healthcare Associated Infections: MRSA lower CL (HAI-5 CI LOWER) (Score),Healthcare Associated Infections: MRSA patient days (HAI-5 DOPC) (Score),Healthcare Associated Infections: MRSA upper CL (HAI-5 CI UPPER) (Score),Healthcare Associated Infections: SSI Abdominal Hysterectomy (SIR) (HAI-4 SIR) (Score),Healthcare Associated Infections: SSI Abdominal Number of Procedures (HAI-4 DOPC) (Score),Healthcare Associated Infections: SSI Abdominal Observed Cases (HAI-4 NUMERATOR) (Score),Healthcare Associated Infections: SSI Abdominal Predicted Cases (HAI-4 ELIG CASES) (Score),Healthcare Associated Infections: SSI Abdominal lower CL (HAI-4 CI LOWER) (Score),Healthcare Associated Infections: SSI Abdominal upper CL (HAI-4 CI UPPER) (Score),Healthcare Associated Infections: SSI Colon Number of Procedures (HAI-3 DOPC) (Score),Healthcare Associated Infections: SSI Colon Observed Cases (HAI-3 NUMERATOR) (Score),Healthcare Associated Infections: SSI Colon Predicted Cases (HAI-3 ELIG CASES) (Score),Healthcare Associated Infections: SSI Colon Surgery (SIR) (HAI-3 SIR) (Score),Healthcare Associated Infections: SSI Colon lower CL (HAI-3 CI LOWER) (Score),Healthcare Associated Infections: SSI Colon upper CL (HAI-3 CI UPP

## Minor formatting

In [9]:
cols = ['State', 'Lon', 'Lat', 'Beds', 'Hospital Ownership', 'Hospital Type', 'file_year', 'file_month', 'Facility Name', 'Facility ID']
for c in cols:
    col = tdf.pop(c)
    tdf.insert(0, col.name, col)

labs = list(tdf)
for c in cols:
    labs.remove(c)
    
print(tdf.shape)
tdf = tdf.dropna(subset=labs, how='all')
print(tdf.shape)
tdf.drop_duplicates(subset=cols, inplace=True)
print(tdf.shape)

tdf.head()

(530242, 379)
(522996, 379)
(219080, 379)


,Facility ID,Facility Name,file_month,file_year,Hospital Type,Hospital Ownership,Beds,Lat,Lon,State,Healthcare Associated Infections: CAUTI (SIR) (HAI-2 SIR) (Score),Healthcare Associated Infections: CAUTI Number of Procedures (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2 NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2a NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2 ELIG CASES) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2a ELIG CASES) (Score),Healthcare Associated Infections: CAUTI SIR (ICUs only) (HAI-2a SIR) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2a DOPC) (Score),Healthcare Associated Infections: CAUTI lower CL (HAI-2 CI LOWER) (Score),Healthcare Associated Infections: CAUTI lower CL (HAI-2a CI LOWER) (Score),Healthcare Associated Infections: CAUTI upper CL (HAI-2 CI UPPER) (Score),Healthcare Associated Infections: CAUTI upper CL (HAI-2a CI UPPER) (Score),Healthcare Associated Infections: CDIFF (SIR) (HAI-6 SIR) (Score),Healthcare Associated Infections: CDIFF Observed Cases (HAI-6 NUMERATOR) (Score),Healthcare Associated Infections: CDIFF Predicted Cases (HAI-6 ELIG CASES) (Score),Healthcare Associated Infections: CDIFF lower CL (HAI-6 CI LOWER) (Score),Healthcare Associated Infections: CDIFF patient days (HAI-6 DOPC) (Score),Healthcare Associated Infections: CDIFF upper CL (HAI-6 CI UPPER) (Score),Healthcare Associated Infections: CLABSI (SIR) (HAI-1 SIR) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1a DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Procedures (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1 NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1a NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1 ELIG CASES) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1a ELIG CASES) (Score),Healthcare Associated Infections: CLABSI SIR (ICUs only) (HAI-1a SIR) (Score),Healthcare Associated Infections: CLABSI lower CL (HAI-1 CI LOWER) (Score),Healthcare Associated Infections: CLABSI lower CL (HAI-1a CI LOWER) (Score),Healthcare Associated Infections: CLABSI upper CL (HAI-1 CI UPPER) (Score),Healthcare Associated Infections: CLABSI upper CL (HAI-1a CI UPPER) (Score),Healthcare Associated Infections: MRSA (SIR) (HAI-5 SIR) (Score),Healthcare Associated Infections: MRSA Observed Cases (HAI-5 NUMERATOR) (Score),Healthcare Associated Infections: MRSA Predicted Cases (HAI-5 ELIG CASES) (Score),Healthcare Associated Infections: MRSA lower CL (HAI-5 CI LOWER) (Score),Healthcare Associated Infections: MRSA patient days (HAI-5 DOPC) (Score),Healthcare Associated Infections: MRSA upper CL (HAI-5 CI UPPER) (Score),Healthcare Associated Infections: SSI Abdominal Hysterectomy (SIR) (HAI-4 SIR) (Score),Healthcare Associated Infections: SSI Abdominal Number of Procedures (HAI-4 DOPC) (Score),Healthcare Associated Infections: SSI Abdominal Observed Cases (HAI-4 NUMERATOR) (Score),Healthcare Associated Infections: SSI Abdominal Predicted Cases (HAI-4 ELIG CASES) (Score),Healthcare Associated Infections: SSI Abdominal lower CL (HAI-4 CI LOWER) (Score),Healthcare Associated Infections: SSI Abdominal upper CL (HAI-4 CI UPPER) (Score),Healthcare Associated Infections: SSI Colon Number of Procedures (HAI-3 DOPC) (Score),Healthcare Associated Infections: SSI Colon Observed Cases (HAI-3 NUMERATOR) (Score),Healthcare Associated Infections: SSI Colon Predicted Cases (HAI-3 ELIG CASES) (Score),Healthcare Associated Infections: SSI Colon Surgery (SIR) (HAI-3 SIR) (Score),Healthcare Associated Infections: SSI Colon lower CL (HAI-3 CI LOWER) (Score),Healthcare Ass

In [41]:
ttdf = tdf.copy(deep=True)
print(ttdf.shape)

ttdf = ttdf[((ttdf['file_month'] == '10') & (ttdf['file_year'].isin(['2020', '2023', '2018', '2021', '2022']))) | ((ttdf['file_month'] == '07') & (ttdf['file_year'].isin(['2013', '2014', '2015', '2016', '2017', '2018', '2019']))) | ((ttdf['file_month'] == '12') & (ttdf['file_year'].isin(['2016'])))] 
print(ttdf.shape)

'''
# Remove columns with more than x% empty values (NaN)
threshold = 0.5 * len(ttdf.index)
ttdf = ttdf.dropna(thresh=threshold, axis=1)
print(ttdf.shape)

# Remove rows with more than x% empty values (NaN)
threshold = 0.5 * len(ttdf.columns)
ttdf = ttdf.dropna(thresh=threshold, axis=0)
print(ttdf.shape)
'''

ls = []
for i in list(ttdf):
    if 'LOWER' in i or 'UPPER' in i or 'lower' in i or 'upper' in i or 'estimate' in i or 'Estimate' in i:
        ls.append(i)

ttdf['Hospital'] = ttdf['Facility Name'] + ' (' + ttdf['Facility ID'] + ')'
ttdf.drop(labels=ls+['file_month', 'Facility ID', 'Facility Name'], axis=1, inplace=True)
col = ttdf.pop('Hospital')
ttdf.insert(0, 'Hospital', col)
print(ttdf.shape)

ttdf.head()

(219080, 379)
(57773, 379)
(57773, 323)


,Hospital,file_year,Hospital Type,Hospital Ownership,Beds,Lat,Lon,State,Healthcare Associated Infections: CAUTI (SIR) (HAI-2 SIR) (Score),Healthcare Associated Infections: CAUTI Number of Procedures (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2 NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2a NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2 ELIG CASES) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2a ELIG CASES) (Score),Healthcare Associated Infections: CAUTI SIR (ICUs only) (HAI-2a SIR) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2a DOPC) (Score),Healthcare Associated Infections: CDIFF (SIR) (HAI-6 SIR) (Score),Healthcare Associated Infections: CDIFF Observed Cases (HAI-6 NUMERATOR) (Score),Healthcare Associated Infections: CDIFF Predicted Cases (HAI-6 ELIG CASES) (Score),Healthcare Associated Infections: CDIFF patient days (HAI-6 DOPC) (Score),Healthcare Associated Infections: CLABSI (SIR) (HAI-1 SIR) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1a DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Procedures (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1 NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1a NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1 ELIG CASES) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1a ELIG CASES) (Score),Healthcare Associated Infections: CLABSI SIR (ICUs only) (HAI-1a SIR) (Score),Healthcare Associated Infections: MRSA (SIR) (HAI-5 SIR) (Score),Healthcare Associated Infections: MRSA Observed Cases (HAI-5 NUMERATOR) (Score),Healthcare Associated Infections: MRSA Predicted Cases (HAI-5 ELIG CASES) (Score),Healthcare Associated Infections: MRSA patient days (HAI-5 DOPC) (Score),Healthcare Associated Infections: SSI Abdominal Hysterectomy (SIR) (HAI-4 SIR) (Score),Healthcare Associated Infections: SSI Abdominal Number of Procedures (HAI-4 DOPC) (Score),Healthcare Associated Infections: SSI Abdominal Observed Cases (HAI-4 NUMERATOR) (Score),Healthcare Associated Infections: SSI Abdominal Predicted Cases (HAI-4 ELIG CASES) (Score),Healthcare Associated Infections: SSI Colon Number of Procedures (HAI-3 DOPC) (Score),Healthcare Associated Infections: SSI Colon Observed Cases (HAI-3 NUMERATOR) (Score),Healthcare Associated Infections: SSI Colon Predicted Cases (HAI-3 ELIG CASES) (Score),Healthcare Associated Infections: SSI Colon Surgery (SIR) (HAI-3 SIR) (Score),Outpatient Imaging Efficiency: Abdomen CT Use of Contrast Material (OP-10) (Score),Outpatient Imaging Efficiency: Breast Cancer Screening Recall Rates (OP-39) (Score),Outpatient Imaging Efficiency: MRI Lumbar Spine for Low Back Pain (OP-8) (Score),Outpatient Imaging Efficiency: Mammography Follow-up Rates (OP-9) (Score),Outpatient Imaging Efficiency: Outpatients who got cardiac imaging stress tests before low-risk outpatient surgery (OP-13) (Score),Outpatient Imaging Efficiency: Outpatients with brain CT scans who got a sinus CT scan at the same time (OP-14) (Score),Outpatient Imaging Efficiency: Thorax CT Use of Contrast Material (OP-11) (Score),Payment and Value of Care: Payment for heart attack patients (PAYM_30_AMI) (Denominator),Payment and Value of Care: Payment for heart attack patients (PAYM_30_AMI) (Payment),Payment and Value of Care: Payment for heart failure patients (PAYM_30_HF) (Denominator),Payment and Value of Care: Payment for heart failure patients (PAYM_30_HF) (Payment),Payment and Value of Care: Payment for hip/knee replacement patients (PAYM_90_HIP_KNEE) (Denominator),Payment and Value of Care: Payment for hip/knee replacement patients (PAY

In [44]:
ls1 = ['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State']

## Make Healthcare Associated Infections files
ls2 = []
for i in list(ttdf):
    if 'Healthcare Associated Infections:' in i:
        if '1a' in i or '2a' in i:
            continue
        else:
            ls2.append(i)

tdf2 = ttdf.filter(items=ls1 + ls2, axis=1)


for i in list(tdf2):
    if 'Healthcare Associated Infections:' in i:
        ni = i.replace('Healthcare Associated Infections: ', '')
        tdf2.rename(columns={i: ni}, inplace=True)
        
        if ' (Score)' in ni:
            ni2 = ni.replace(' (Score)', '')
            tdf2.rename(columns={ni: ni2}, inplace=True)

            
tdf2.rename(columns={'CAUTI (SIR) (HAI-2 SIR)': 'CAUTI (SIR)',
                     'CAUTI Number of Procedures (HAI-2 DOPC)': 'CAUTI Number of Procedures',
                     'CAUTI Observed Cases (HAI-2 NUMERATOR)': 'CAUTI Observed Cases',
                     'CAUTI Predicted Cases (HAI-2 ELIG CASES)': 'CAUTI Predicted Cases',
                     'CAUTI Urinary Catheter Days (HAI-2 DOPC)': 'CAUTI Urinary Catheter Days',
                     'CDIFF (SIR) (HAI-6 SIR)': 'CDIFF (SIR)',
                     'CDIFF Observed Cases (HAI-6 NUMERATOR)': 'CDIFF Observed Cases',
                     'CDIFF Predicted Cases (HAI-6 ELIG CASES)': 'CDIFF Predicted Cases',
                     'CDIFF patient days (HAI-6 DOPC)': 'CDIFF patient days',
                     'CLABSI (SIR) (HAI-1 SIR)': 'CLABSI (SIR)',
                     'CLABSI Number of Device Days (HAI-1 DOPC)': 'CLABSI Number of Device Days',
                     'CLABSI Number of Procedures (HAI-1 DOPC)': 'CLABSI Number of Procedures',
                     'CLABSI Observed Cases (HAI-1 NUMERATOR)': 'CLABSI Observed Cases',
                     'CLABSI Predicted Cases (HAI-1 ELIG CASES)': 'CLABSI Predicted Cases',
                     'MRSA (SIR) (HAI-5 SIR)': 'MRSA (SIR)',
                     'MRSA Observed Cases (HAI-5 NUMERATOR)': 'MRSA Observed Cases',
                     'MRSA Predicted Cases (HAI-5 ELIG CASES)': 'MRSA Predicted Cases',
                     'MRSA patient days (HAI-5 DOPC)': 'MRSA patient days',
                     'SSI Abdominal Hysterectomy (SIR) (HAI-4 SIR)': 'SSI Abdominal Hysterectomy',
                     'SSI Abdominal Number of Procedures (HAI-4 DOPC)': 'SSI Abdominal Number of Procedures',
                     'SSI Abdominal Observed Cases (HAI-4 NUMERATOR)': 'SSI Abdominal Observed Cases',
                     'SSI Abdominal Predicted Cases (HAI-4 ELIG CASES)': 'SSI Abdominal Predicted Cases',
                     'SSI Colon Number of Procedures (HAI-3 DOPC)': 'SSI Colon Number of Procedures',
                     'SSI Colon Observed Cases (HAI-3 NUMERATOR)': 'SSI Colon Observed Cases',
                     'SSI Colon Predicted Cases (HAI-3 ELIG CASES)': 'SSI Colon Predicted Cases',
                     'SSI Colon Surgery (SIR) (HAI-3 SIR)': 'SSI Colon Surgery (SIR)',
                    }, inplace=True)
print(tdf2.shape)
tdf2.head()

(57773, 34)
['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State', 'CAUTI (SIR)', 'CAUTI Number of Procedures', 'CAUTI Observed Cases', 'CAUTI Predicted Cases', 'CAUTI Urinary Catheter Days', 'CDIFF (SIR)', 'CDIFF Observed Cases', 'CDIFF Predicted Cases', 'CDIFF patient days', 'CLABSI (SIR)', 'CLABSI Number of Device Days', 'CLABSI Number of Procedures', 'CLABSI Observed Cases', 'CLABSI Predicted Cases', 'MRSA (SIR)', 'MRSA Observed Cases', 'MRSA Predicted Cases', 'MRSA patient days', 'SSI Abdominal Hysterectomy', 'SSI Abdominal Number of Procedures', 'SSI Abdominal Observed Cases', 'SSI Abdominal Predicted Cases', 'SSI Colon Number of Procedures', 'SSI Colon Observed Cases', 'SSI Colon Predicted Cases', 'SSI Colon Surgery (SIR)']


,Hospital,file_year,Hospital Type,Hospital Ownership,Beds,Lat,Lon,State,CAUTI (SIR),CAUTI Number of Procedures,CAUTI Observed Cases,CAUTI Predicted Cases,CAUTI Urinary Catheter Days,CDIFF (SIR),CDIFF Observed Cases,CDIFF Predicted Cases,CDIFF patient days,CLABSI (SIR),CLABSI Number of Device Days,CLABSI Number of Procedures,CLABSI Observed Cases,CLABSI Predicted Cases,MRSA (SIR),MRSA Observed Cases,MRSA Predicted Cases,MRSA patient days,SSI Abdominal Hysterectomy,SSI Abdominal Number of Procedures,SSI Abdominal Observed Cases,SSI Abdominal Predicted Cases,SSI Colon Number of Procedures,SSI Colon Observed Cases,SSI Colon Predicted Cases,SSI Colon Surgery (SIR)
40,SOUTHEAST ALABAMA MEDICAL CENTER (010001),2014,Acute Care,Government - Hospital District or Authority,377.0,31.214058,-85.361725,AL,2.409,NaN,22.0,9.133,7611.0,0.569,32.0,56.217,73385.0,1.378,2901.0,NaN,6.0,4.353,0.741,4.0,5.401,74564.0,NaN,68.0,0.0,0.743,144.0,3.0,4.746,0.632
44,SOUTHEAST ALABAMA MEDICAL CENTER (010001),2015,Acute Care,Government - Hospital District or Authority,377.0,31.214058,-85.361725,AL,3.464,NaN,31.0,8.949,7450.0,0.850,58.0,68.217,96420.0,1.818,2199.0,NaN,6.0,3.300,1.391,10.0,7.187,99435.0,NaN,58.0,1.0,0.713,171.0,4.0,5.868,0.682
46,SOUTHEAST ALABAMA MEDICAL CENTER (010001),2017,Acute Care,Government - Hospital District or Authority,377.0,31.214058,-85.361725,AL,NaN,NaN,NaN,NaN,NaN,0.296,21.0,70.848,89328.0,NaN,NaN,NaN,NaN,NaN,1.979,9.0,4.548,89328.0,0.0,211.0,0.0,1.901,155.0,2.0,4.342,0.461
49,SOUTHEAST ALABAMA MEDICAL CENTER (010001),2018,Acute Care,Government - Hospital District or Authority,377.0,31.214058,-85.361725,AL,0.827,NaN,11.0,13.308,13501.0,0.672,49.0,72.932,99655.0,1.097,8000.0,NaN,7.0,6.382,0.871,5.0,5.743,100032.0,0.0,251.0,0.0,2.326,184.0,3.0,5.216,0.575
52,SOUTHEAST ALABAMA MEDICAL CENTER (010001),2019,Acute Care,Government - Hospital District or Authority,377.0,31.214058,-85.361725,AL,1.082,NaN,16.0,14.786,13269.0,0.430,30.0,69.766,94830.0,1.457,7283.0,NaN,9.0,6.179,0.178,1.0,5.633,95739.0,0.0,210.0,0.0,1.809,171.0,0.0,4.763,0.000


## Write HAI file date specific files

In [45]:
years = sorted(tdf2['file_year'].unique())

for yr in years:
    tdf3 = tdf2[tdf2['file_year'] == yr]
    tdf3.reset_index(drop=True, inplace=True)
    tdf3.to_csv(main_dir + 'dataframes/filtered_files/HAIs/HAIs_' + yr + '.csv')

## Load and Merge Measurement Dates files

In [ ]:
ls2 = ['HVBP',
       'Outpatient Imaging Efficiency',
       'Payment and Value of Care',
       'Complications and Deaths',
       'Timely and Effective Care',
       'Hospital Readmissions Reduction Program',
       'Hospital Acquired Conditions Reduction Program',
       'HCAHPS',
       'Unplanned Visits',
]

In [ ]:
df = pd.read_csv(main_dir + 'measure_dates/hvbp_measure_dates.csv')
print('df.shape:', df.shape)
df['Measure Name'] = ls2[0] + ': ' + df['Measure Name']


ls1 = ['hvbp_measure_dates_df.csv',
       'Outpatient_Imaging_Efficiency_df.csv', 
       'Payment_and_ValueOfCare_df.csv', 
       'complication_and_deaths_df.csv', 
       'Timely_and_Effective_Care_df.csv',
       'hrrp_df.csv',
       'hacrp_df.csv',
       'hcahps_df.csv',
       'unplanned_visits_df.csv',
      ]

brk = 0
for i, l in enumerate(ls1):
    if i == 0:
        continue
        
    print(l)
    tdf = pd.read_csv(main_dir + 'measure_dates/' + l)
    tdf['Measure Name'] = ls2[i] + ': ' + tdf['Measure Name']
    
    df = pd.concat([df, tdf])
    print('df.shape:', df.shape)
    
    for i in list(df):
        if '_x' in i:
            print('redundant column name:', i)
            brk = 1
            break
            
    if brk == 1: break
    del tdf

df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
print('df.shape:', df.shape)
df.head()


In [ ]:
df.to_csv('~/GitHub/hospitals-data-archive/measure_dates/all_measure_dates.csv')


In [39]:
url = 'https://github.com/klocey/HAIs/raw/main/1_data/WinsorizedZscores.pkl'
winz_df = pd.read_pickle(url)

print(winz_df.shape)
winz_df.head()

(212892, 50)


,index,Facility and File Date,Predicted Cases,Days,SIR,upper CL,lower CL,Observed Cases,file date,provider,date,O/E,simulated O,simulated O/E,expected O,expected O/E,pi_opt,z_opt,HAI,exp_random,SISc,SISo,SIS,Observed is better than random,Winzorized SIR,Winzorized SIS,Winzorized SISc,Winzorized SISo,Winzorized z SIR,Winzorized z SIS,Winzorized z SISc,Winzorized z SISo,Winzorized z SISo rank,Winzorized z SIS rank,Winzorized z SISc rank,Winzorized z SIR rank,CLABSI Number of Procedures,CAUTI Number of Procedures,weight,all better than random,weighted Winzorized z SIS,weighted Winzorized z SISo,weighted Winzorized z SISc,weighted Winzorized z SIR,total HAI SIS,total HAI SISo,total HAI SISc,total HAI SIR,sqrt_Days,avg days
212112,52281,010001-20200422,18.459,12696,0.271,0.600,0.099,5.0,2020-04-22,010001,20200422,0.270871,5,0.270871,12.184349,0.660076,0.001340,5035.623680,CAUTI,15.120709,-3.031704,-0.389206,-0.389206,1,0.271,-0.389206,-3.031704,-0.389206,-0.910484,-0.811561,-0.588670,-0.811561,536.0,536.0,377.0,434.0,NaN,NaN,0.25,0,-0.097301,-0.097301,-0.757926,-0.227621,-0.114357,-0.114357,4.030116,-0.357149,112.676528,204.602984
111794,70848,010001-20200422,69.709,96978,0.430,0.607,0.296,30.0,2020-04-22,010001,20200422,0.430360,35,0.502087,40.295451,0.578052,0.000460,10394.980775,CDIFF,45.636843,-0.649582,-0.147692,-0.147692,1,0.430,-0.147692,-0.649582,-0.147692,-0.456042,-0.453652,-0.180364,-0.453652,1086.0,1086.0,1093.0,1042.0,NaN,NaN,0.25,0,-0.036923,-0.036923,-0.162396,-0.114011,-0.114357,-0.114357,4.030116,-0.357149,311.412909,204.602984
159384,45669,010001-20200422,7.122,6874,1.123,2.133,0.522,8.0,2020-04-22,010001,20200422,1.123280,5,0.702050,4.198512,0.589513,0.000964,3977.845504,CLABSI,5.922490,1.731966,0.533767,0.533767,0,1.123,0.533767,1.731966,0.533767,0.811675,0.840932,0.128864,0.840932,1635.0,1635.0,1295.0,1613.0,NaN,NaN,0.25,0,0.133442,0.133442,0.432991,0.202919,-0.114357,-0.114357,4.030116,-0.357149,82.909589,204.602984
39919,39919,010001-20200422,6.895,96978,0.290,0.958,0.049,2.0,2020-04-22,010001,20200422,0.290065,3,0.435098,5.132368,0.744361,0.000095,76628.116155,MRSA,7.038988,34.995986,-0.454296,-0.454296,1,0.290,-0.454296,18.069784,-0.454296,-0.873747,-0.787647,2.679417,-0.787647,427.0,427.0,1639.0,363.0,NaN,NaN,0.25,0,-0.113574,-0.113574,4.517446,-0.218437,-0.114357,-0.114357,4.030116,-0.357149,311.412909,204.602984
212383,52552,010005-20200422,4.803,7844,1.457,2.883,0.637,7.0,2020-04-22,010005,20200422,1.457422,8,1.665626,6.403081,1.333142,0.001340,5035.623680,CAUTI,8.531876,0.410814,0.124280,0.124280,0,1.457,0.124280,0.410814,0.124280,1.367108,0.159908,0.012335,0.159908,1383.0,1383.0,1019.0,1985.0,NaN,NaN,0.25,0,0.031070,0.031070,0.102704,0.341777,0.076828,0.076828,1.175709,0.594473,88.566359,135.126539
